# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [73]:
import pandas as pd
import numpy as np
import warnings

from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [61]:
df = pd.read_csv('previsao_de_renda.csv')

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [63]:
df.drop('data_ref', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_cliente             15000 non-null  int64  
 1   sexo                   15000 non-null  object 
 2   posse_de_veiculo       15000 non-null  bool   
 3   posse_de_imovel        15000 non-null  bool   
 4   qtd_filhos             15000 non-null  int64  
 5   tipo_renda             15000 non-null  object 
 6   educacao               15000 non-null  object 
 7   estado_civil           15000 non-null  object 
 8   tipo_residencia        15000 non-null  object 
 9   idade                  15000 non-null  int64  
 10  tempo_emprego          12427 non-null  float64
 11  qt_pessoas_residencia  15000 non-null  float64
 12  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(5)
memory usage: 1.3+ MB


In [65]:
# Convertendo int64 para float
df['id_cliente'] = df['id_cliente'].astype(float)
df['qtd_filhos'] = df['qtd_filhos'].astype(float)
df['idade'] = df['idade'].astype(float)

# Convertendo bool para float
df['posse_de_veiculo'] = df['posse_de_veiculo'].astype(float)
df['posse_de_imovel'] = df['posse_de_imovel'].astype(float)

# Convertendo object para float usando codificação de rótulos
label_cols = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
encoder = LabelEncoder()
for col in label_cols:
    df[col] = encoder.fit_transform(df[col]).astype(float)

# Verificando as transformações
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_cliente             15000 non-null  float64
 1   sexo                   15000 non-null  float64
 2   posse_de_veiculo       15000 non-null  float64
 3   posse_de_imovel        15000 non-null  float64
 4   qtd_filhos             15000 non-null  float64
 5   tipo_renda             15000 non-null  float64
 6   educacao               15000 non-null  float64
 7   estado_civil           15000 non-null  float64
 8   tipo_residencia        15000 non-null  float64
 9   idade                  15000 non-null  float64
 10  tempo_emprego          12427 non-null  float64
 11  qt_pessoas_residencia  15000 non-null  float64
 12  renda                  15000 non-null  float64
dtypes: float64(13)
memory usage: 1.5 MB
None


In [66]:
df.dropna(inplace=True)

# Verificando as transformações
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_cliente             12427 non-null  float64
 1   sexo                   12427 non-null  float64
 2   posse_de_veiculo       12427 non-null  float64
 3   posse_de_imovel        12427 non-null  float64
 4   qtd_filhos             12427 non-null  float64
 5   tipo_renda             12427 non-null  float64
 6   educacao               12427 non-null  float64
 7   estado_civil           12427 non-null  float64
 8   tipo_residencia        12427 non-null  float64
 9   idade                  12427 non-null  float64
 10  tempo_emprego          12427 non-null  float64
 11  qt_pessoas_residencia  12427 non-null  float64
 12  renda                  12427 non-null  float64
dtypes: float64(13)
memory usage: 1.3 MB
None


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [67]:
# Definindo X e y
X = df.drop(columns=['renda'])
y = df['renda']

# Dividindo os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Verificando o tamanho dos conjuntos
print(f"Conjunto de treinamento (X): {X_train.shape}")
print(f"Conjunto de teste (X): {X_test.shape}")
print(f"Conjunto de treinamento (y): {y_train.shape}")
print(f"Conjunto de teste (y): {y_test.shape}")


Conjunto de treinamento (X): (9320, 12)
Conjunto de teste (X): (3107, 12)
Conjunto de treinamento (y): (9320,)
Conjunto de teste (y): (3107,)


In [68]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results = {}

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    
    # Previsões no conjunto de teste
    y_pred = ridge.predict(X_test)
    
    # Calcula o R^2 no conjunto de teste
    r2 = r2_score(y_test, y_pred)
    
    results[alpha] = r2

# Imprime os resultados
for alpha, r2 in results.items():
    print(f"Alpha: {alpha}, R^2: {r2:.4f}")


Alpha: 0, R^2: 0.2960
Alpha: 0.001, R^2: 0.2960
Alpha: 0.005, R^2: 0.2960
Alpha: 0.01, R^2: 0.2960
Alpha: 0.05, R^2: 0.2960
Alpha: 0.1, R^2: 0.2960


In [69]:
# Suprimir avisos
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Definindo os valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Armazenando os resultados
results = {}

# Loop através de cada valor de alpha
for alpha in alphas:
    # Inicializando e treinando o modelo LASSO
    lasso = Lasso(alpha=alpha, max_iter=10000)  # Defini max_iter para garantir a convergência
    lasso.fit(X_train, y_train)
    
    # Prevendo valores para o conjunto de teste
    y_pred = lasso.predict(X_test)
    
    # Avaliando R^2 para o conjunto de teste
    r2 = r2_score(y_test, y_pred)
    results[alpha] = r2

# Exibindo os resultados
for alpha, r2 in results.items():
    print(f"Alpha: {alpha} - R^2: {r2}")


Alpha: 0 - R^2: 0.29597765522086394
Alpha: 0.001 - R^2: 0.2959776880648749
Alpha: 0.005 - R^2: 0.29597781944341284
Alpha: 0.01 - R^2: 0.2959779836107841
Alpha: 0.05 - R^2: 0.2959792963535781
Alpha: 0.1 - R^2: 0.2959809348064504


In [70]:
def forward_stepwise_selection(X_train, y_train, X_test, y_test):
    remaining_features = list(X_train.columns)
    current_features = []
    best_r2 = 0.0
    best_new_feature = None

    while remaining_features:
        r2_with_new_feature = []

        for feature in remaining_features:
            model = LinearRegression()
            model.fit(X_train[current_features + [feature]], y_train)
            predictions = model.predict(X_test[current_features + [feature]])
            r2 = r2_score(y_test, predictions)
            r2_with_new_feature.append((r2, feature))

        r2_with_new_feature.sort(reverse=True)
        best_r2_new, best_new_feature = r2_with_new_feature[0]

        if best_r2_new > best_r2:
            remaining_features.remove(best_new_feature)
            current_features.append(best_new_feature)
            best_r2 = best_r2_new
        else:
            break

    return current_features, best_r2

selected_features, best_r2 = forward_stepwise_selection(X_train, y_train, X_test, y_test)
print("Melhores recursos selecionados:", selected_features)
print("Melhor R^2:", best_r2)


Melhores recursos selecionados: ['tempo_emprego', 'sexo', 'idade', 'educacao', 'qt_pessoas_residencia', 'tipo_renda', 'posse_de_imovel', 'estado_civil', 'id_cliente', 'tipo_residencia', 'qtd_filhos']
Melhor R^2: 0.2965460941712518


In [71]:
# 1. Engenharia de Características: Criação de características polinomiais
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# 5. Normalização dos dados
scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

# Treinando novamente o modelo LASSO (como um exemplo) com características polinomiais e dados normalizados
lasso = Lasso(alpha=0.05)  
lasso.fit(X_train_poly_scaled, y_train)
predictions = lasso.predict(X_test_poly_scaled)
r2 = r2_score(y_test, predictions)
print("R^2 com características polinomiais e dados normalizados:", r2)


R^2 com características polinomiais e dados normalizados: 0.40305682495784867


In [72]:
# Criando o modelo
tree_regressor = DecisionTreeRegressor()

# Treinando o modelo
tree_regressor.fit(X_train, y_train)

# Fazendo predições na base de testes
y_pred = tree_regressor.predict(X_test)

# Avaliando o R^2
r2_tree = r2_score(y_test, y_pred)
print(f"R^2 da árvore de regressão: {r2_tree:.4f}")

# Experimentando com diferentes profundidades
for depth in range(1, 15):
    tree_regressor = DecisionTreeRegressor(max_depth=depth)
    tree_regressor.fit(X_train, y_train)
    y_pred = tree_regressor.predict(X_test)
    r2_tree = r2_score(y_test, y_pred)
    print(f"R^2 da árvore de regressão com profundidade {depth}: {r2_tree:.4f}")


R^2 da árvore de regressão: 0.3808
R^2 da árvore de regressão com profundidade 1: 0.1018
R^2 da árvore de regressão com profundidade 2: 0.2503
R^2 da árvore de regressão com profundidade 3: 0.3135
R^2 da árvore de regressão com profundidade 4: 0.3498
R^2 da árvore de regressão com profundidade 5: 0.3712
R^2 da árvore de regressão com profundidade 6: 0.3354
R^2 da árvore de regressão com profundidade 7: 0.3449
R^2 da árvore de regressão com profundidade 8: 0.3035
R^2 da árvore de regressão com profundidade 9: 0.3306
R^2 da árvore de regressão com profundidade 10: 0.3374
R^2 da árvore de regressão com profundidade 11: 0.3270
R^2 da árvore de regressão com profundidade 12: 0.3333
R^2 da árvore de regressão com profundidade 13: 0.3659
R^2 da árvore de regressão com profundidade 14: 0.3635
